## AoC 2024 - Day 16

In [ ]:
#from icecream import ic
import heapq

data = open('test.txt').read().split('\n')

# #Same, but using a dictionary, with keys being (row, column) tuples
# #Makes in/out of range checks dead simple, using grid.get((r,c))
# #    None if it doesn't exist, value otherwise
grid = {(r,c): char for r, row in enumerate(data) for c, char in enumerate(row)}

def astar(grid, start, goal, dir):
    open_set = []

    # Priority queue with (F-score, node)
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == goal:
            # Reconstruct the path and return
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path
        
        for dr, dc, dir_new in [(0, 1, ">"), (0, -1, "<"), (1, 0, "^"), (-1, 0, "v")]:
            r, c = current[0] + dr, current[1] + dc

            neighbor = (r, c)

            # Assuming uniform cost for each step
            tentative_g = g_score[current] + 1

            if dir != dir_new:
                tentative_g += 1000
            
            if not grid.get(neighbor): continue
            if grid.get(neighbor) == "#": continue

            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    g_score[neighbor] = tentative_g
                    dist = abs(neighbor[0] - goal[0]) + abs(neighbor[1] - goal[1]) 
                    f_score = tentative_g + dist
                    heapq.heappush(open_set, (f_score, neighbor))
                    came_from[neighbor] = current
    
    return None  # No path found

start = (15, 1)
goal = (1, 15)
direction = ">"

print(grid[start], grid[goal])
#path = astar(grid, start, goal)
#print("Shortest path:", path)


S E
